In [ ]:
import requests
import json
import pandas as pd

### Anwendungsbeispiel 1: Automatisierte Übersetzung mit DeepL

Read the [docs](https://www.deepl.com/docs-api)

In [ ]:
# Alternative 1 (empfohlen): Abruf Auth Token aus externer Datei
with open('token.json') as f:
    credentials = json.load(f)
    AUTH_KEY_DEEPL = credentials['AUTH_TOKEN_DEEPL']

# Alternative 2 (nicht empfohlen): Manuelle Eingabe des Auth Token
# AUTH_KEY_DEEPL = '<AUTH TOKEN>'

In [ ]:
ENDPOINT = 'https://api-free.deepl.com/v2/translate'

text = """Data literacy is the ability to read, understand, create, and communicate data as information. Much like literacy as a general concept, data literacy focuses on the competencies involved in working with data.
It is, however, not similar to the ability to read text since it requires certain skills involving reading and understanding data."""

HEADER = {'Authorization': 'DeepL-Auth-Key ' + AUTH_KEY_DEEPL}

PARAMS = {'source_lang': 'EN',
          'target_lang': 'DE',
          'text': text}

response = requests.get(ENDPOINT, headers=HEADER, params=PARAMS)

In [ ]:
response

In [ ]:
response.json()

In [ ]:
print(response.json()['translations'][0]['text'])

In [ ]:
def translate(text, source_lang, target_lang):
        
    ENDPOINT = 'https://api-free.deepl.com/v2/translate'
    
    with open('token.json') as f:
        credentials = json.load(f)
        AUTH_KEY_DEEPL = credentials['AUTH_TOKEN_DEEPL']

    # AUTH_KEY_DEEPL = '<AUTH TOKEN>'
    
    HEADER = {'Authorization': 'DeepL-Auth-Key ' + AUTH_KEY_DEEPL}

    PARAMS = {'source_lang': source_lang,
              'target_lang': target_lang,
              'text': text}
    
    response = requests.get(ENDPOINT, headers=HEADER, params=PARAMS)
    
    return response.json()['translations'][0]['text']

In [ ]:
sentences = ["Data literacy is the ability to read, understand, create, and communicate data as information.",
             "Much like literacy as a general concept, data literacy focuses on the competencies involved in working with data.",
             "It is, however, not similar to the ability to read text since it requires certain skills involving reading and understanding data."]

for sent in sentences:
    print(translate(sent, 'EN', 'DE'))

*Hinweis: Es gibt sogar ein separates Python package (`deepl`), was die Benutzung der API noch einfacher macht!*

### Anwendungsbeispiel 2: Transkription mit AssemblyAI

Read the [docs](https://docs.assemblyai.com/)

Schritt 1: Upload der Audio-Datei

In [ ]:
UPLOAD_ENDPOINT = 'https://api.assemblyai.com/v2/upload'

with open('token.json') as f:
    credentials = json.load(f)
    AUTH_TOKEN_ASSEMBLYAI = credentials['AUTH_TOKEN_ASSEMBLYAI']

# AUTH_TOKEN_ASSEMBLYAI = '<AUTH TOKEN>'

HEADER = {'authorization': AUTH_TOKEN_ASSEMBLYAI,
          'content-type': 'application/json'}

with open('data/recording.m4a', 'rb') as f:
    audio_file = f.read()
    
response = requests.post(UPLOAD_ENDPOINT, headers=HEADER, data=audio_file)
response

In [ ]:
response.json()

In [ ]:
UPLOAD_URL = response.json()['upload_url']

Schritt 2: Starten der Transkription

Inzwischen werden auch Transkriptionen in [Spanisch, Französisch, Deutsch und Italienisch](https://docs.assemblyai.com/#supported-languages) unterstützt.

In [ ]:
TRANSCRIPT_ENDPOINT = 'https://api.assemblyai.com/v2/transcript'

JSON = {'audio_url': UPLOAD_URL,
        'language_code': 'en',
        'speaker_labels': 'true'}

response = requests.post(TRANSCRIPT_ENDPOINT, headers=HEADER, json=JSON)
response

In [ ]:
response.json()

In [ ]:
TRANSCRIPT_ID = response.json()['id']

Schritt 3: Download des Transkriptes

*Hinweis: Laut [Anbieter](https://docs.assemblyai.com/#processing-times) veranschlagt der Prozess etwa 30% der Länge der Audio-Datei.*

In [ ]:
response = requests.get(TRANSCRIPT_ENDPOINT + '/' + TRANSCRIPT_ID, headers=HEADER)
response

In [ ]:
response.json()

In [ ]:
response.json()['text']

### Anwendungsbeispiel 3: Chatbots

Read the [docs](https://platform.openai.com/docs/introduction)

Verwendung mittels `requests` Bibliothek

In [ ]:
# Alternative 1 (empfohlen): Abruf Auth Token aus externer Datei
with open('token.json') as f:
    credentials = json.load(f)
    AUTH_KEY_OPENAI = credentials['OPENAI_API_KEY']

# Alternative 2 (nicht empfohlen): Manuelle Eingabe des Auth Token
# AUTH_KEY_OPENAI = '<AUTH TOKEN>'

In [ ]:
ENDPOINT = 'https://api.openai.com/v1/chat/completions'

text = """
APIs, kurz für Application Programming Interfaces, sind schwer zu verstehen.
Erkläre in einfachen Worten was eine API ist und wie sie funktioniert. Nutze Metaphern und Analogien, um es verständlicher zu machen.
"""

HEADER = {'Authorization': 'Bearer ' + AUTH_KEY_OPENAI}

MESSAGE = {'model': 'gpt-3.5-turbo',
           'messages': [{'role': 'user', 'content': text}]}

response = requests.post(ENDPOINT, headers=HEADER, json=MESSAGE)

In [ ]:
response

In [ ]:
response.json()

In [ ]:
print(response.json()['choices'][0]['message']['content'])

Verwendung mittels `openai` Bibliothek

In [ ]:
import openai

openai.api_key = AUTH_KEY_OPENAI

MESSAGE = {'model': 'gpt-3.5-turbo',
           'messages': [{'role': 'user', 'content': text}]}

response = openai.ChatCompletion.create(
    model='gpt-3.5-turbo',
    messages=MESSAGE['messages'],
    temperature=0
)

In [ ]:
response

In [ ]:
print(response['choices'][0]['message']['content'])

Ein Praktischer Anwendungsfall: News Summarization

In [ ]:
with open('data/news_article.txt') as f:
    news = f.read()

print(news)

In [ ]:
text = f"""
Perform the following actions step-by-step:
Step 1 - Summarize the key information in the news article delimited by triple dashes. Use three sentences at most.
Step 2 - Extract the names of all entities (e.g., companies, countries, persons) mentioned in the article.
Step 3 - Is the article discussing a rating action by Moody's?


Structure your answer using the following format:
Step 1: <list of bullet points>
Step 2: <list of bullet points>
Step 3: <yes/no as boolean value>

 
News article:
---
Body: {news}
---
"""

print(text)

In [ ]:
MESSAGE = {'model': 'gpt-3.5-turbo',
           'messages': [{'role': 'user', 'content': text}]}

response = openai.ChatCompletion.create(
    model='gpt-3.5-turbo',
    messages=MESSAGE['messages'],
    temperature=0
)

In [ ]:
print(response['choices'][0]['message']['content'])

### Anwendungsbeispiel 4: Amtliche Statistiken mit Datenguide

Read the [docs](https://datengui.de/docs)

*Hinweis: Da Datenguide open data bereitstellt brauchen wir keine `auth_token`!

Der `data` Code `52411:ISV001` kennzeichnet den Datensatz "Statistik über beantragte Insolvenzverfahren - Insolvenzverfahren", der `region` Code `05` steht für "Nordrhein-Westfalen", und es werden alle verfügbaren Daten ab `2008` abgerufen.*

In [ ]:
ENDPOINT_DATENGUIDE = 'https://tabular.genesapi.org'

PARAMS = {'data': '52411:ISV001',
          'region': '05',
          'time': '2008:',
          'format':'json'}

response = requests.get(ENDPOINT_DATENGUIDE, params=PARAMS)
response

In [ ]:
response.json()

In [ ]:
response.json()['data']

In [ ]:
df = pd.DataFrame(response.json()['data'])
df

In [ ]:
df.drop_duplicates(subset=['year', 'value']).plot.line(x='year', y='value')